I'm Hadi here, new on kaggle and Data Science. I'm using data set having path ../input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv. for sure it is not collected by me. I'm just using it for learning purpose. i'm thankful to one who has collected this. regards: Hadi Bux

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/all-trumps-twitter-insults-20152021'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# read csv file and save it in df
df = pd.read_csv('../input/all-trumps-twitter-insults-20152021/trump_insult_tweets_2014_to_2021.csv')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# show top five rows of dataset
df.head()

we have 5 columns. **unnamed: 0** column which is just like index, **target** which has person name, **insult** which is particular word used for insult and **tweet** which represents actuall tweet

we see that column named Unnamed: 0 is unnecessary, we will delete it

In [ ]:
# deleting unnecessary column
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# check data type of columns
df.dtypes

In [ ]:
missing_data = df.isnull()
missing_data.head()

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

In [ ]:
# index 2533 and 7439 have target values null
print(df[df['target'].isnull()])

In [ ]:
target_value_counts = df['target'].value_counts()
target_value_counts

there 866 different targets 

# get list all mentions in tweets

make function which concatinates strings if tweet contains more than 1 mentions

In [ ]:
def concat(s):
    temp = ''
    i = 0
    for value in s:
        temp = temp +' '+ value
    return temp

create mention list

In [ ]:
m = re.compile(r'@\w+')
#mention_list = m.finditer(df['tweet'].to_json())
mention_list = []
temp = '' # temporary variable for storing list of finds
for value in df['tweet']:
    temp = m.findall(str(value))
    if temp:
        if len(temp) > 1: # if temp is greater than 1
            mention_list.append(concat(temp)) # call concat function
        else: # if list contains 1 item
            mention_list.append(temp[0])
    else:
        mention_list.append('None')
# print first 50 mentions
for i in range(50):
    print(mention_list[i])


In [ ]:
# letls make column of all mentions
mention_array = np.array(mention_list)
df['mentions']= pd.Series(mention_array)
df['mentions'].value_counts().head(50)

most tweets don't have mentions 7927. there are 558 unique mentions and @nytimes is at top with value of 236 followed by CNN. note: some tweets contain more than 2 mentions. value_count() function count them as another unique value

In [ ]:
some_mentions = ['@nytimes', '@CNN', '@FoxNews', '@foxandfriends', '@JebBush'] # top 5 mentions
counts = [236, 179, 172, 75, 43] # seats in assembly
explode=(0.2, 0.2, 0.2, 0.2, 0.2) # distance from each other
pie=plt.pie(counts,
        explode=explode,
        labels=some_mentions,
        autopct='%.1f%%',
        shadow=True,
        startangle=45
)
plt.axis('equal')
plt.title('Pie Graph of top 5 mentions')
plt.legend(pie[0], some_mentions, loc=2)
plt.show()

# Sentiment Analysis of Tweets 

In [ ]:
# import library
from textblob import TextBlob

In [ ]:
polarity = list()
for value in df['tweet']:
    value
    if TextBlob(value).sentiment.polarity > 0:
        polarity.append("Positive")
    elif TextBlob(value).sentiment.polarity < 0:
        polarity.append("Negative")
    else:
        polarity.append("Neutral")    

In [ ]:
df['sentiment'] = polarity

In [ ]:
df[['tweet', 'sentiment']].head(10)

In [ ]:
df['sentiment'].value_counts()

**it seems sarcasim does not have good relation with textblob libray. it can't detect sarcastic text**

# Visualization

In [ ]:
df.head()

In [ ]:
import seaborn as sns
sns.countplot(x = df['sentiment'])
plt.show()

#  Commonly used keywords

In [ ]:
# required modules
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
tokens = []
stop_words = stopwords.words("english")
# GET TEXT FROM DATAFRAME
for tweet in df['tweet']:
# MAKE TOKENS OF TEXT AFTER CONVERTING IT TO LOWERCASE
    #text = tweet
    word_tokens = word_tokenize(tweet.lower())
    # ITERATE OVER TOKEN
    for word in word_tokens:
        # IF WORD IS ALPHABETICAL
        if word.isalpha(): # IGNORE ALL HASHTAGS 
            # IF WORD IS NOT IN STOPWORDS LIST
            if word not in stop_words:
                tokens.append( word)
fwords = []
countwords = Counter(tokens)
#release_list(tokens), it takes 5 more seconds 
for word in countwords.most_common(50):
    fwords.append(word[0] + ':' + " used " + str(word[1]) + ' times')

In [ ]:
# top 50 most used words
fwords

# 